In [1]:
import os

BASE_URL = f"http://localhost:3000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:3000/v1


In [2]:
!curl http://localhost:3000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

{"object":"list","data":[{"id":"/home/user/Models/deepseek-ai/deepseek-llm-7b-chat","object":"model","created":1752925351,"owned_by":"vllm","root":"/home/user/Models/deepseek-ai/deepseek-llm-7b-chat","parent":null,"max_model_len":2048,"permission":[{"id":"modelperm-1a1f2d6fe4de47ca96d063fef7aed1f5","object":"model_permission","created":1752925351,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [3]:
!pip install -q pydantic_ai openai     

In [35]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url="http://localhost:3000/v1",  # Your vLLM server
    api_key="dummy",  # vLLM doesn't require real API key
)
agent_model = OpenAIModel("/home/user/Models/deepseek-ai/deepseek-llm-7b-chat", provider=provider)

In [36]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model
)

In [37]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


In [42]:
from datetime import datetime
from mcp.server.fastmcp import FastMCP
app = FastMCP()
app.tool()
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [43]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [44]:
@app.tool()
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [49]:
agent = Agent(
    model=agent_model,
    toolsets=[app],
    system_prompt = (
        "You have access to:\n"
        "   1. get_current_time()\n"
        
    )
)

In [50]:
await run_async("What’s the date today?")

AttributeError: 'FastMCP' object has no attribute 'apply'